In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

In [8]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.layers import Flatten, Dense
from skimage.transform import resize
from keras.layers import Dropout

# Load the MobileNet model pre-trained on ImageNet data
base_model = MobileNet(
    weights='imagenet', include_top=False, input_shape=(96, 96, 3))
x = base_model.output
x = Flatten()(x)  # Flatten the output layer to one dimension
# x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Function to preprocess the dataset


def preprocess_data(X, target_size):
    # Reshape and resize the data
    X_reshaped = np.reshape(X.values, (-1, 48, 48))
    X_resized = np.array([resize(img, (target_size, target_size))
                         for img in X_reshaped])
    X_rgb = np.repeat(X_resized[..., np.newaxis], 3, axis=3)

    # Preprocess the input for MobileNet
    X_preprocessed = preprocess_input(X_rgb)
    return X_preprocessed


# Load your dataset
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")["0"]

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")["0"]

# Preprocess the data
X_train_preprocessed = preprocess_data(X_train, 96)
X_test_preprocessed = preprocess_data(X_test, 96)

# Convert y to one-hot encoding
# Adjust 'num_classes' based on your dataset
y_train_encoded = to_categorical(y_train, num_classes=10)
# Adjust 'num_classes' based on your dataset
y_test_encoded = to_categorical(y_test, num_classes=10)

# Train the model
model.fit(X_train_preprocessed, y_train_encoded, epochs=10, batch_size=64)

# Evaluate the model
predictions = model.predict(X_test_preprocessed)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == y_test)

print(f'Accuracy: {accuracy}')

Epoch 1/10


2023-11-23 15:21:59.112442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 16s 86ms/step - loss: 0.4548 - accuracy: 0.8975
Epoch 2/10
152/152 [==============================] - 12s 79ms/step - loss: 0.0664 - accuracy: 0.9822
Epoch 3/10
152/152 [==============================] - 12s 77ms/step - loss: 0.0231 - accuracy: 0.9938
Epoch 4/10
152/152 [==============================] - 12s 79ms/step - loss: 0.0201 - accuracy: 0.9949
Epoch 5/10
152/152 [==============================] - 12s 80ms/step - loss: 0.0291 - accuracy: 0.9936
Epoch 6/10
152/152 [==============================] - 12s 80ms/step - loss: 0.0256 - accuracy: 0.9942
Epoch 7/10
152/152 [==============================] - 12s 78ms/step - loss: 0.0309 - accuracy: 0.9921
Epoch 8/10
152/152 [==============================] - 12s 78ms/step - loss: 0.0540 - accuracy: 0.9877
Epoch 9/10
152/152 [==============================] - 12s 78ms/step - loss: 0.0266 - accuracy: 0.9946
Epoch 10/10
152/152 [==============================] - 12s 78ms/step - loss: 0.0191 - accurac

2023-11-23 15:24:02.003635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


97/97 [==============================] - 2s 14ms/step
Accuracy: 0.9870550161812298


## Here we got 98.7% accuracy using pretrained model MobileNet

### Using the same model, but with synthetic data

In [7]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.layers import Flatten, Dense
from skimage.transform import resize
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

# Data Augmentation setup
datagen = ImageDataGenerator(
    rotation_range=20,      # Rotate images by up to 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2, # Shift images vertically by up to 20% of the height
    shear_range=0.2,        # Shear images by up to 20 degrees
    zoom_range=0.2,         # Zoom in on images by up to 20%
    horizontal_flip=True,   # Allow horizontal flipping
    fill_mode='nearest'     # Fill in newly created pixels
)

# Load the MobileNet model pre-trained on ImageNet data
base_model = MobileNet(
    weights='imagenet', include_top=False, input_shape=(96, 96, 3))
x = base_model.output
x = Flatten()(x)  # Flatten the output layer to one dimension
x = Dropout(0.5)(x)  # Add dropout for regularization
# Adjust '10' to the number of classes in your dataset
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Function to preprocess the dataset


def preprocess_data(X, target_size):
    # Reshape and resize the data
    X_reshaped = np.reshape(X.values, (-1, 48, 48))
    X_resized = np.array([resize(img, (target_size, target_size))
                         for img in X_reshaped])
    X_rgb = np.repeat(X_resized[..., np.newaxis], 3, axis=3)

    # Preprocess the input for MobileNet
    X_preprocessed = preprocess_input(X_rgb)
    return X_preprocessed


# Load your dataset
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")["0"]

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")["0"]

# Preprocess the data
X_train_preprocessed = preprocess_data(X_train, 96)
X_test_preprocessed = preprocess_data(X_test, 96)

# Convert y to one-hot encoding
# Adjust 'num_classes' based on your dataset
y_train_encoded = to_categorical(y_train, num_classes=10)
# Adjust 'num_classes' based on your dataset
y_test_encoded = to_categorical(y_test, num_classes=10)

# Fit the model with data augmentation
model.fit(datagen.flow(X_train_preprocessed, y_train_encoded, batch_size=64),
          steps_per_epoch=len(X_train_preprocessed) / 64, epochs=10)

# Evaluate the model
predictions = model.predict(X_test_preprocessed)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == y_test)

print(f'Accuracy: {accuracy}')

Epoch 1/10


2023-11-23 14:12:53.018846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


151/151 [==============================] - 15s 84ms/step - loss: 1.2285 - accuracy: 0.7469
Epoch 2/10
151/151 [==============================] - 12s 78ms/step - loss: 0.2437 - accuracy: 0.9298
Epoch 3/10
151/151 [==============================] - 12s 78ms/step - loss: 0.1397 - accuracy: 0.9580
Epoch 4/10
151/151 [==============================] - 12s 78ms/step - loss: 0.1245 - accuracy: 0.9608
Epoch 5/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0956 - accuracy: 0.9722
Epoch 6/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0994 - accuracy: 0.9716
Epoch 7/10
151/151 [==============================] - 12s 78ms/step - loss: 0.1085 - accuracy: 0.9734
Epoch 8/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0921 - accuracy: 0.9749
Epoch 9/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0946 - accuracy: 0.9756
Epoch 10/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0650 - accurac

2023-11-23 14:14:55.187241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


97/97 [==============================] - 2s 14ms/step
Accuracy: 0.9647249190938512
